In [28]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from util import *
from STMatrix import STMatrix
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
adj = pd.read_csv("data/gy_contest_link_top.txt", delimiter=';', dtype={'in_links': np.str, 'out_links': str})
adj = adj.fillna('')

info = pd.read_csv("data/gy_contest_link_info.txt", delimiter=';')

In [ ]:
adj.head()

In [ ]:
links = adj.link_ID
link2idx = dict(zip(links, range(len(links))))

In [ ]:
link2idx

In [19]:
f = file('link.adjlist', 'w')
for idx, row in adj.iterrows():
    if row['out_links'] == '':
        continue
    line = [row['link_ID']] + row['out_links'].split('#')
    ids = [str(l) for l in line]
    f.write(' '.join(ids) + "\n")
f.close()

In [22]:
embeddings = file.readlines(file('link.embeddings', 'r'))
link_embeddings = {}
for line in embeddings[1:]:
    fields = line.strip().split(' ')
    link = fields[0]
    embedding = [float(v) for v in fields[1:]]
    link_embeddings[link] = embedding
link_embeddings

{'3377906280028510514': [1.835477,
  -0.377694,
  -3.022821,
  0.940285,
  0.673926,
  0.450192,
  2.153312,
  2.214557],
 '3377906280395510514': [3.020393,
  -0.370407,
  -1.204561,
  1.008517,
  -2.593346,
  -4.234977,
  -0.639027,
  0.72393],
 '3377906281518510514': [-3.226248,
  0.731184,
  -2.351949,
  -2.843736,
  -2.97048,
  -1.257937,
  -0.015032,
  -0.253032],
 '3377906281774510514': [0.984119,
  -1.508292,
  -4.378376,
  -0.692332,
  0.131636,
  0.788416,
  -0.548791,
  -0.154879],
 '3377906282328510514': [3.33196,
  -0.343985,
  -2.062845,
  1.401671,
  -2.677282,
  -2.896473,
  -1.091879,
  1.16891],
 '3377906282418510514': [1.401078,
  2.2499,
  -0.694863,
  -1.626725,
  -2.17546,
  -1.115243,
  2.054798,
  4.16881],
 '3377906283328510514': [2.94436,
  -0.36196,
  -1.390494,
  1.171558,
  -2.412049,
  -3.357592,
  -0.797332,
  0.74145],
 '3377906284028510514': [-0.617713,
  2.18571,
  -3.274845,
  -0.56932,
  3.898216,
  0.267544,
  0.066502,
  -0.234562],
 '33779062843955

In [34]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy
from scipy import spatial
links = link_embeddings.keys()
similarities = np.zeros((len(links), len(links)))
for i, li in enumerate(links):
    for j, lj in enumerate(links):
        similarities[i, j] = spatial.distance.cosine(link_embeddings[li], link_embeddings[lj])

In [ ]:
# plot correlation matrix
plt.rc('figure', figsize=(40,40))
fig = plt.figure() #调用figure创建一个绘图对象

ax = fig.add_subplot(111)
cax = ax.matshow(similarities, cmap='Blues', vmin=-1, vmax=1)  #绘制热力图，从-1到1
fig.colorbar(cax)  #将matshow生成热力图设置为颜色渐变条
ticks = numpy.arange(len(links)) #生成0-9，步长为1
ax.set_xticks(ticks)  #生成刻度
ax.set_yticks(ticks)
ax.set_xticklabels(range(len(links))) #生成x轴标签
ax.set_yticklabels(range(len(links)))
plt.savefig('embedding.png')

In [40]:
from IPython.display import Image, FileLink
FileLink('embedding.png')

/home/zhli7/guizhou/embedding.png

In [41]:
similarities

array([[  2.22044605e-16,   9.42506342e-01,   8.68709629e-01, ...,
          9.40918828e-01,   8.47554752e-01,   9.20147641e-01],
       [  9.42506342e-01,  -2.22044605e-16,   1.12627428e+00, ...,
          9.81852660e-01,   1.06212805e+00,   5.95949457e-01],
       [  8.68709629e-01,   1.12627428e+00,   1.11022302e-16, ...,
          5.84528086e-01,   6.38774904e-01,   8.88206049e-01],
       ..., 
       [  9.40918828e-01,   9.81852660e-01,   5.84528086e-01, ...,
          0.00000000e+00,   1.69018318e-01,   1.29205493e+00],
       [  8.47554752e-01,   1.06212805e+00,   6.38774904e-01, ...,
          1.69018318e-01,  -2.22044605e-16,   1.17784790e+00],
       [  9.20147641e-01,   5.95949457e-01,   8.88206049e-01, ...,
          1.29205493e+00,   1.17784790e+00,   2.22044605e-16]])

In [45]:
links[0], links[13], links[21]

('4377906287869500514', '3377906281518510514', '4377906287169500514')

In [44]:
adj[adj.link_ID==links[0]]

,link_ID,in_links,out_links
34,4377906287869500514,4377906288869500514,4377906284759500514#3377906281518510514


In [49]:
links.index('4377906284759500514')

71

In [51]:
import cPickle as pickle
pickle.dump(link_embeddings, open('link_embeddings.plk', 'wb'))